In [1]:
from pathlib import Path
import pdb
import os
import torch
import cv2
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import shutil
import json
%matplotlib inline

In [3]:
from Bezier import bezier_curve
import math

# 参数
# Pyten-20201022
save_path = "/home/pantengteng/datasets/bdd100k"
# save_path = "/home/pantengteng/datasets/bdd100k/generated_labels/100k/train" # save_path = "/home/pantengteng/datasets/bdd100k/seg"
ignore_list = ["road curb", "crosswalk"]
show_img = False #True # True
save_img = True #False #False #  False
GRID_NUM = 100

# 画 点
def draw_point(img, lane_pt, text='', line_color=(255,0,0), text_color=(0, 0, 0)):
    # pdb.set_trace()
    for i in range(len(lane_pt)):
        pt = lane_pt[i]
        # 画圆，圆心为：(160, 160)，半径为：60，颜色为：point_color，实心线
        # cv2.circle(img, (int(pt[0]), int(pt[1])), 3, (0,0,255), 0)
        if text:
            cv2.putText(img, text + "_%d"%i, (int(pt[0]), int(pt[1])), cv2.FONT_HERSHEY_COMPLEX, 0.5, text_color, 1)
        
        # 画线
        if i < len(lane_pt) - 1:
            next_pt = lane_pt[i + 1]
            cv2.line(img, (int(pt[0]), int(pt[1])), (int(next_pt[0]), int(next_pt[1])), line_color, 8, 4)
            
    
    return img

#计算两线距离
# 只计算起止点，双线的端点顺序可能相反，需要判断
# 需要将点的顺序修正，方便后续平均--->sorted->sort（sorted新建list）
def dist(pset1, pset2):
    set2_down_pt = pset2[-1]
    set2_up_pt = pset2[0]
    set1_down_pt = pset1[-1]
    set1_up_pt = pset1[0]
    
    dis1 = math.sqrt((set1_down_pt[0] - set2_down_pt[0])**2 + (set1_down_pt[1] - set2_down_pt[1])**2)
    dis2 = math.sqrt((set1_up_pt[0] - set2_up_pt[0])**2 + (set1_up_pt[1] - set2_up_pt[1])**2)
    max_dis = max(dis1, dis2)
    return max_dis

# 计算直线距离
def cal_angle(vec1, vec2):
    vec1_norm = math.sqrt(vec1[0]**2 + vec1[1]**2)
    vec2_norm = math.sqrt(vec2[0]**2 + vec2[1]**2)
    vec_dot = vec1[0]*vec2[0] + vec1[1]*vec2[1]
    try:
        cos = vec_dot / (vec1_norm * vec2_norm)
    except Exception as e:
        pdb.set_trace()
    degree = np.arccos(cos) * 180 / np.pi
    return degree

# 融合
# v1:平均：仅适用于点数相同的情况；
# v2:拟合
# def add(pset1, pset2):
#     r_set = []
#     for i in range(len(pset1)):
#         r_set.append([(pset1[i][0] + pset2[i][0]) / 2, (pset1[i][1] + pset2[i][1]) / 2])
#     return r_set

#                 #if lane_pt[0][2] != "C":
#                 #    print("More than 3 points but not Bezier curve!")
#                 #    sp_case += 1
#                 xvals, yvals = bezier_curve(lane_pt, nTimes=1000)
#                 print(len(xvals))
#                 lane_pt = [(xvals[i],yvals[i]) for i in range(len(xvals))]

# def add(obj1, obj2):
#     pset1 = obj1["poly2d"]
#     pset2 = obj2["poly2d"]
#     if pset1[0][2] == "C" or pset1[-1][2] == "C" or pset2[0][2] == "C" or pset2[-1][2] == "C":
#         print("Bezier curve!")
#         xvals1, yvals1 = bezier_curve(pset1, nTimes=100)
#         xvals2, yvals2 = bezier_curve(pset2, nTimes=100)
#         r_set = [[(xvals1[i] + xvals2[i]) / 2, (yvals1[i] + yvals2[i]) / 2] for i in range(len(xvals1))]
#     else:
#         if len(obj1["poly2d"]) == len(obj2["poly2d"]):
#             print("equal num", len(obj1["poly2d"]))
#             r_set = []
#             for i in range(len(pset1)):
#                 r_set.append([(pset1[i][0] + pset2[i][0]) / 2, (pset1[i][1] + pset2[i][1]) / 2])
#     #         if pset1[0][2] == "C":
#     #             print("equal and Bezier curve!")
#     #             xvals, yvals = bezier_curve(r_set, nTimes=1000)
#     #             print(len(xvals))
#     #             r_set = [(xvals[i],yvals[i]) for i in range(len(xvals))]
#             else:
#                 print("Not equal, not Bezier curve!")
#                 r_set = [[(pset1[0][0] + pset2[0][0]) / 2, (pset1[0][1] + pset2[0][1]) / 2],
#                           [(pset1[-1][0] + pset2[-1][0]) / 2, (pset1[-1][1] + pset2[-1][1]) / 2]]
#     return r_set

def add(obj1, obj2):
    pset1 = obj1["poly2d"]
    pset2 = obj2["poly2d"]
#     if pset1[0][2] == "C" or pset1[-1][2] == "C" and pset2[0][2] == "C" or pset2[-1][2] == "C":
#         print("Bezier curve!")
#         xvals1, yvals1 = bezier_curve(pset1, nTimes=100)
#         xvals2, yvals2 = bezier_curve(pset2, nTimes=100)
#         r_set = [[(xvals1[i] + xvals2[i]) / 2, (yvals1[i] + yvals2[i]) / 2] for i in range(len(xvals1))]
#     else:
        
    if len(pset1) == len(pset2):
        # print("equal num", len(obj1["poly2d"]), "curve mode", pset1[0][2])
        r_set = []
        for i in range(len(pset1)):
            r_set.append([(pset1[i][0] + pset2[i][0]) / 2, (pset1[i][1] + pset2[i][1]) / 2])
            
    elif pset1[0][2] == "L" and pset1[-1][2] == "L" and pset2[0][2] == "L" and pset2[-1][2] == "L":
        # print("Not equal, not Bezier curve!")
        r_set = [[(pset1[0][0] + pset2[0][0]) / 2, (pset1[0][1] + pset2[0][1]) / 2],
                  [(pset1[-1][0] + pset2[-1][0]) / 2, (pset1[-1][1] + pset2[-1][1]) / 2]]
    else:
        # 若C则BC化，所以这里只剩一种：即一L，一C
        # print("mix L and C")
        if len(pset1) < len(pset2):
            pset1, pset2 = pset2, pset1
            
        x_arr = np.linspace(pset2[0][0], pset2[-1][0], num=GRID_NUM, endpoint=True)
        y_arr = np.linspace(pset2[0][1], pset2[-1][1], num=GRID_NUM, endpoint=True)

        r_set = []
        for i in range(len(pset1)):
            r_set.append([(pset1[i][0] + x_arr[i]) / 2, (pset1[i][1] + y_arr[i]) / 2])
        
    return r_set

def maybe_same_lane(cur_lane, another_lane):
    if ( cur_lane["category"] == another_lane["category"] and
            cur_lane["attributes"] == another_lane["attributes"] ):
            # cur_lane["id"] == another_lane["id"] + 1 and 
            # dist(cur_lane["poly2d"], another_lane["poly2d"]) < 100 ):
        # print(cur_lane["id"], another_lane["id"])
        return True
    else:
        return False

# 注意：会改变原始的列表
def combine_same_lane(cur_lane, another_lane):
    lane_pt = add(cur_lane, another_lane)
    cur_lane["poly2d"] = lane_pt
    return cur_lane
  
# Pyten-20201022
with open(save_path + "/val.txt", "r") as f:
# with open(save_path + "/seg_lane.txt", "r") as f:
    all_list = f.readlines()

max_lane_num = 0
total_num = 0
no_lane_num = 0
sp_case = 0
for img_name in all_list:
    # pdb.set_trace()
    img_name = img_name.strip("\n")
    print(img_name)
    # img_name = "51450dbd-7d259a47"#"1e2d42a4-b1b0abcc"#"2ad1aa4b-9852e784"#"3dab6efd-ecb3083b"#"5c60ec08-4fbeeb70"#"1aa6dd36-7ca1967a"#"4c5c24ad-d0338ef4"#"0b2867ef-a6ca437b"#"0cdde16a-e2d9677e" #"4f45d5ce-d87a28f1"
    if Path(f"{save_path}/generated_labels/100k/val/{img_name}.png").exists():
        print("exist!")
        continue
    # Pyten-20201022
    img = cv2.imread(save_path + "/images/100k/val/" + img_name + ".jpg") #, cv2.IMREAD_GRAYSCALE)
    # img = cv2.imread(save_path + "/seg_images/" + img_name + ".jpg") #, cv2.IMREAD_GRAYSCALE)
    #img1 = img.copy()
    
    # Pyten-20201022
    label_file = save_path + "/labels/100k/val/" + img_name + ".json"
    #label_file = save_path + "/lane_labels/" + img_name + ".json"
    with open(label_file, encoding="utf-8") as f:
         label_dic = json.load(f)
    
    # 筛选出需要的lane
    same_line_flg = False
    objs = label_dic["frames"][0]["objects"]
    obj_index = 0
    lane_list = []
    while obj_index < len(objs):
        # pdb.set_trace()
        obj = objs[obj_index]
        if "lane" in obj["category"]:
            # 过滤车道线类型
            if (obj["category"][5:] in ignore_list) or (obj["attributes"]["direction"] != "parallel"):
                obj_index += 1
                continue
            # 按照y更新点的顺序，起点在远端
            lane_pt = obj["poly2d"]

            # BC曲线控制点转换为点集
            if len(lane_pt) > 2:
                # pdb.set_trace()
                if lane_pt[0][2] != "C" and lane_pt[-1][2] != "C":
                    print("More than 3 points but not Bezier curve!")
                    print(obj["id"])
                    sp_case += 1
                # print("Convert Bezier curve!")
                xvals, yvals = bezier_curve(lane_pt, nTimes=GRID_NUM)
                obj["poly2d"] = [[xvals[i], yvals[i], "C"] for i in range(len(xvals))]

            obj["poly2d"].sort(key=lambda x:x[1])
        
            #计算与水平方向的夹角
            angle = cal_angle((obj["poly2d"][-1][0] - obj["poly2d"][0][0], obj["poly2d"][-1][1] - obj["poly2d"][0][1]), (10, 0))

            if angle < 5 or angle > 175:
                # print("horizontal %d"%obj["id"])
                obj_index += 1
                continue
                
            # Pyten-20201022
            #img1 = draw_point(img1, obj["poly2d"], str(obj["id"]), line_color=(0, 255, 0))
        
            lane_list.append(obj)
            
        obj_index += 1
    
    # 合并双线
    combined_lane_list = []
    lane_index = 0
    while lane_index < len(lane_list):
        left_index = lane_index + 1
        min_distance = 100 # float('inf')
        min_index = lane_index
        # 每条线只能融合一次，最小
        while left_index < len(lane_list):
            cur_lane = lane_list[lane_index]
            another_lane = lane_list[left_index]
            if maybe_same_lane(cur_lane, another_lane):
                # pdb.set_trace()
                cur_distance = dist(cur_lane["poly2d"], another_lane["poly2d"])
                if min_distance > cur_distance:
                    min_distance = cur_distance
                    min_index = left_index
            left_index += 1
        if min_index != lane_index:
            lane_list[lane_index] = combine_same_lane(lane_list[lane_index], lane_list[min_index])
            # print("combine {} and {}".format(lane_list[lane_index]["id"], lane_list[min_index]["id"]))
            lane_list.pop(min_index)

        combined_lane_list.append(lane_list[lane_index])
        lane_index += 1
    
    lane_num = len(combined_lane_list)
    total_num += lane_num
    if lane_num == 0:
        no_lane_num += 1
    else:
        with open(save_path + "/all_with_lane.txt", "a") as f:
            f.write(img_name + "\n")
    print("lane_num", lane_num)
    if max_lane_num < lane_num:
        max_lane_num = lane_num
    
    # Pyten-20201013-SetLaneNumTo4
    # 根据近端y坐标排序
    combined_lane_list = sorted(combined_lane_list, key=lambda x:x["poly2d"][-1][1], reverse=True)
    combined_lane_list = combined_lane_list[:4]
    
    lane_num = len(combined_lane_list)
    print("reduced_lane_num", lane_num)
    
    # 重新排列
    # 依据远端点x坐标进行排序
    combined_lane_list = sorted(combined_lane_list, key=lambda x:x["poly2d"][0][0])
    
    # 可视化
    # 转换label
    label = np.zeros((img.shape[0], img.shape[1]))
    # pdb.set_trace()
    
    for idx,lane in enumerate(combined_lane_list):
        # Pyten-20201022
        #img = draw_point(img, lane["poly2d"], str(idx), line_color=(0,0,255 ))
        label = draw_point(label, lane["poly2d"], line_color=idx+1)
    
# 合并同线，由于情况不多，暂时放弃拟合
#     while lane_index < len(lane_list) - 1:
#         left_index = lane_index + 1
#         while left_index < len(lane_list):
#             cur_lane = lane_list[lane_index]
#             another_lane = lane_list[left_index]
#             if same_lane(cur_lane, another_lane):
#                 lane_list.pop(left_index)
#                 lane_list[lane_index] = combine_same_lane(cur_lane, another_lane)
    
    if save_img:
        # pdb.set_trace()
        # converted_img_10/
        # cv2.imwrite(f"{save_path}/convert_test_4lane/{img_name}.jpg", img)
        # Pyten-20201022
        cv2.imwrite(f"{save_path}/generated_labels/100k/val/{img_name}.png", label)
        #cv2.imwrite(f"{save_path}/generated_labels_4lane/{img_name}.png", label)
        
    if show_img:
        img = img[:,:,::-1]
        img1 = img1[:,:,::-1]
        plt.figure(figsize=(20,10))
        # plt.imshow(img)
        # plt.imshow(img1)
        # plt.show()
        plt.subplot(1,2,1)
        plt.imshow(img1)
        plt.axis("off")

        plt.subplot(1,2,2)
        plt.imshow(img)
        plt.axis("off")
        plt.show()
print("max_lane_num", max_lane_num)
print("avg_num", total_num/len(all_list))
print("no_lane_num", no_lane_num)
print("sp_case:", sp_case)
#out_label = np.zeros((img.shape[0],img.shape[1]))
# c5e41dc2-537a34c6

b8781f69-76c72b31
lane_num 5
reduced_lane_num 4
b4a8ec73-1c6196a0
lane_num 4
reduced_lane_num 4
c12ce21f-6f6ecfa7
lane_num 1
reduced_lane_num 1
b633ed08-08a8b4ce
lane_num 0
reduced_lane_num 0
c5c86b04-083cb3fd
lane_num 7
reduced_lane_num 4
bfaa2229-ae426591
lane_num 0
reduced_lane_num 0
b73282b1-7bc49185
lane_num 4
reduced_lane_num 4
c776d10b-a1314ad4
lane_num 1
reduced_lane_num 1
b83ec6be-953a634b
lane_num 1
reduced_lane_num 1
c78f7d9c-783ee02d
lane_num 2
reduced_lane_num 2
c61e0f0d-188e2618
lane_num 2
reduced_lane_num 2
bdba96db-510677bf
lane_num 5
reduced_lane_num 4
bc05da8e-e0eda9bd
lane_num 3
reduced_lane_num 3
c6758a15-d4e2e5ac
lane_num 4
reduced_lane_num 4
b6b047b4-3fccc761
lane_num 0
reduced_lane_num 0
b2e54795-0a58624f
lane_num 0
reduced_lane_num 0
bcbb5f4d-93c5f35d
lane_num 0
reduced_lane_num 0
c159a842-b245f317
lane_num 0
reduced_lane_num 0
bd1ae0c2-a649baaa
lane_num 1
reduced_lane_num 1
b4b9c39d-6409d1fd
lane_num 2
reduced_lane_num 2
b96d19af-4b6a5a63
lane_num 4
reduced_lan

lane_num 3
reduced_lane_num 3
c703904e-4aae94cd
lane_num 0
reduced_lane_num 0
c81ff2c0-6243618f
lane_num 2
reduced_lane_num 2
b57bfcbd-60692014
lane_num 6
reduced_lane_num 4
bcbf4834-88a0b878
lane_num 2
reduced_lane_num 2
c909f3df-d3ce29f1
lane_num 0
reduced_lane_num 0
b5764fb7-c47fa3ec
lane_num 2
reduced_lane_num 2
bfbe2ad2-ec5dea9d
lane_num 1
reduced_lane_num 1
bb8dba97-c7ec77a3
lane_num 4
reduced_lane_num 4
c37d47c0-c830ec1c
lane_num 2
reduced_lane_num 2
c2186a76-6757a4be
lane_num 1
reduced_lane_num 1
bedb9474-8dd30e6f
lane_num 5
reduced_lane_num 4
ba87390f-22dabe83
lane_num 1
reduced_lane_num 1
bcef7536-5c2e5a49
lane_num 0
reduced_lane_num 0
b657d595-3f1fa3ca
lane_num 6
reduced_lane_num 4
bfee82ef-f5aad313
lane_num 0
reduced_lane_num 0
c57cda5b-92fa0083
lane_num 0
reduced_lane_num 0
bdd853b8-4b3664f3
lane_num 1
reduced_lane_num 1
b3039ada-2ef2a725
lane_num 3
reduced_lane_num 3
c1fa7950-56a78519
lane_num 0
reduced_lane_num 0
bd1b8b79-ba219f33
lane_num 2
reduced_lane_num 2
bb8b844a-e

c420e95a-f7f8eacd
lane_num 1
reduced_lane_num 1
c721de81-d9e0ad96
lane_num 5
reduced_lane_num 4
bc6dd79a-fa684758
lane_num 1
reduced_lane_num 1
b1d968b9-563405f4
lane_num 3
reduced_lane_num 3
c3436372-eb18934d
lane_num 0
reduced_lane_num 0
bc88c0cd-a78224ad
lane_num 2
reduced_lane_num 2
c6424a42-272aa782
lane_num 0
reduced_lane_num 0
bd512863-2e6735a4
lane_num 3
reduced_lane_num 3
beb12388-2a415a61
lane_num 5
reduced_lane_num 4
ba7b9df0-a0b062ab
lane_num 0
reduced_lane_num 0
c94b896e-40638bf9
lane_num 2
reduced_lane_num 2
b7fd2970-e46faf78
lane_num 3
reduced_lane_num 3
c90ecab0-68dc9839
lane_num 4
reduced_lane_num 4
c0625a26-8bacfe16
lane_num 2
reduced_lane_num 2
c223e6d7-e1a9bb5a
lane_num 7
reduced_lane_num 4
b5a50c2d-0c1d06e0
lane_num 1
reduced_lane_num 1
c68dcb6b-c1775bef
lane_num 3
reduced_lane_num 3
ba70dfe3-4cfe0efb
lane_num 4
reduced_lane_num 4
b255cd6c-f2f28aff
lane_num 0
reduced_lane_num 0
c45e3dc0-3517e269
lane_num 2
reduced_lane_num 2
b39fe3cd-2a77c651
lane_num 2
reduced_lan

c972204c-e7d86629
lane_num 1
reduced_lane_num 1
c1a026fd-769c0a00
lane_num 2
reduced_lane_num 2
b38fb69e-99642683
lane_num 3
reduced_lane_num 3
babc7a6c-c5941835
lane_num 0
reduced_lane_num 0
b8da2353-61f13dfb
lane_num 2
reduced_lane_num 2
c7b41787-4e83c33d
lane_num 3
reduced_lane_num 3
b3b9ee6e-6d2617d1
lane_num 2
reduced_lane_num 2
b28aac6c-9f1b99cb
lane_num 5
reduced_lane_num 4
c8b5bc59-ad1aa597
lane_num 4
reduced_lane_num 4
b87366af-3fb3389e
lane_num 2
reduced_lane_num 2
c3df354d-407d442c
lane_num 0
reduced_lane_num 0
b5780de7-94313c5f
lane_num 3
reduced_lane_num 3
bceeedca-f0bf8606
lane_num 0
reduced_lane_num 0
b726c429-726588e2
lane_num 3
reduced_lane_num 3
b216243d-ad4306b9
lane_num 0
reduced_lane_num 0
c1f74dbc-c774031f
lane_num 4
reduced_lane_num 4
c0e1fd2a-45156dec
lane_num 3
reduced_lane_num 3
bc2a1273-18d05a08
lane_num 2
reduced_lane_num 2
b5e387a7-e3a60891
lane_num 5
reduced_lane_num 4
c664605e-44460227
lane_num 0
reduced_lane_num 0
c389631f-a4b2d909
lane_num 4
reduced_lan

lane_num 5
reduced_lane_num 4
c8ddb7b2-8c453f30
lane_num 5
reduced_lane_num 4
c733594f-0aa2baf5
lane_num 1
reduced_lane_num 1
bd656105-db296383
lane_num 1
reduced_lane_num 1
b242f6b2-99d2f2c1
lane_num 0
reduced_lane_num 0
bdafde0d-03216895
lane_num 0
reduced_lane_num 0
c3cb6d3b-bd44323e
lane_num 0
reduced_lane_num 0
b89aded3-23603d87
lane_num 3
reduced_lane_num 3
ca551156-5999d6d4
lane_num 1
reduced_lane_num 1
b1d7b3ac-5af8623b
lane_num 0
reduced_lane_num 0
ba4432a9-5439b6db
lane_num 1
reduced_lane_num 1
c18d3ffa-2e847265
lane_num 5
reduced_lane_num 4
bd5d35a0-1357b0da
lane_num 4
reduced_lane_num 4
c684b9be-e666905f
lane_num 0
reduced_lane_num 0
bb6264e8-e8dfd14d
lane_num 3
reduced_lane_num 3
b3a7b21a-48bcf2b8
lane_num 0
reduced_lane_num 0
be651ffd-0bc74ac7
lane_num 2
reduced_lane_num 2
b91eae9e-c8fb98e5
lane_num 1
reduced_lane_num 1
beff1d1c-f80d0deb
lane_num 0
reduced_lane_num 0
c422771c-ecd74810
lane_num 3
reduced_lane_num 3
bcf9cd99-5a125838
lane_num 3
reduced_lane_num 3
c55ab634-3

...

lane_num 3
reduced_lane_num 3
c9a653e3-533efdd4
lane_num 1
reduced_lane_num 1
b20234fd-822029be
lane_num 1
reduced_lane_num 1
b76792cb-0b0d99c9
lane_num 0
reduced_lane_num 0
c933ade8-68159c26
lane_num 2
reduced_lane_num 2
c5a178bf-637d1750
lane_num 1
reduced_lane_num 1
c317c38b-ce5550bb
lane_num 1
reduced_lane_num 1
b953efc6-455bdde1
lane_num 4
reduced_lane_num 4
b995df4d-4bf935df
lane_num 4
reduced_lane_num 4
b9fb5382-36e88642
lane_num 0
reduced_lane_num 0
bb8dba97-9f4540b7
lane_num 8
reduced_lane_num 4
bdcd6f24-0b1af2bf
lane_num 5
reduced_lane_num 4
bc7e80ed-0011e5a7
lane_num 2
reduced_lane_num 2
c0c23a94-f2f15c15
lane_num 0
reduced_lane_num 0
b82f91d7-a21bb033
lane_num 2
reduced_lane_num 2
c1be4f15-060de5f9
lane_num 1
reduced_lane_num 1
b3899bec-df46f67b
lane_num 0
reduced_lane_num 0
c86ef161-6b77193e
lane_num 1
reduced_lane_num 1
c8e895be-9bf55449
lane_num 3
reduced_lane_num 3
b7a9b166-341c551a
lane_num 5
reduced_lane_num 4
bff4bb8b-8a96fd04
lane_num 4
reduced_lane_num 4
c82cf5fc-4

lane_num 1
reduced_lane_num 1
c606dc56-2a64e82c
lane_num 5
reduced_lane_num 4
c8fddab0-9ea6d001
lane_num 8
reduced_lane_num 4
b7d7713a-a58eef67
lane_num 5
reduced_lane_num 4
baf24e00-7742132f
lane_num 1
reduced_lane_num 1
b6c2a409-b0dacc13
lane_num 0
reduced_lane_num 0
ba4fbdd7-43dccdf9
lane_num 3
reduced_lane_num 3
c31aa969-cfd19d38
lane_num 4
reduced_lane_num 4
c2ccdd66-06c44d2f
lane_num 3
reduced_lane_num 3
c37b9726-caffaa65
lane_num 4
reduced_lane_num 4
bf2a6720-a23c2b5e
lane_num 1
reduced_lane_num 1
b74d1efd-b4b4590f
lane_num 0
reduced_lane_num 0
c65fad1e-d790b8ce
lane_num 2
reduced_lane_num 2
b77826c3-b22372a9
lane_num 2
reduced_lane_num 2
bd3bd3ab-c51f41b3
lane_num 3
reduced_lane_num 3
c0882fd0-cb7eb3dc
lane_num 0
reduced_lane_num 0
ca23fea0-97c90df7
lane_num 2
reduced_lane_num 2
b28cfc9b-224431ab
lane_num 3
reduced_lane_num 3
c1807c66-2813073f
lane_num 3
reduced_lane_num 3
b1ff4656-0435391e
lane_num 1
reduced_lane_num 1
b1cebfb7-284f5117
lane_num 2
reduced_lane_num 2
b29806ee-e

lane_num 2
reduced_lane_num 2
b841230d-fada32c4
lane_num 3
reduced_lane_num 3
bca84cbc-800427b6
lane_num 1
reduced_lane_num 1
c3cd6c82-21bd6ca4
lane_num 14
reduced_lane_num 4
c15e64e5-f459ff4d
lane_num 1
reduced_lane_num 1
b5058691-1fb39daf
lane_num 3
reduced_lane_num 3
c55b474f-f39c51a8
lane_num 3
reduced_lane_num 3
c8a5a379-cb506f54
lane_num 2
reduced_lane_num 2
c96d4996-d78b1b88
lane_num 2
reduced_lane_num 2
bb8f6c9a-e2c6dc5a
lane_num 1
reduced_lane_num 1
b332df78-28a61de0
lane_num 1
reduced_lane_num 1
ba7ad79b-d9c40465
lane_num 2
reduced_lane_num 2
b2a60634-0817a6eb
lane_num 2
reduced_lane_num 2
bbc19191-6e9a792b
lane_num 6
reduced_lane_num 4
c179d309-59d431e7
lane_num 3
reduced_lane_num 3
b7979b09-89a5a8ea
lane_num 5
reduced_lane_num 4
bc24f222-f198b643
lane_num 0
reduced_lane_num 0
c5b80ec1-7fc5097c
lane_num 5
reduced_lane_num 4
b931fe93-f0b441ba
lane_num 0
reduced_lane_num 0
b4065fc4-ede06556
lane_num 2
reduced_lane_num 2
b68a000f-9a8091f9
lane_num 3
reduced_lane_num 3
bc24f222-

lane_num 7
reduced_lane_num 4
b47262a4-3c664e48
lane_num 1
reduced_lane_num 1
c06d23aa-28bc6026
lane_num 1
reduced_lane_num 1
bcde71de-fcf3da5c
lane_num 8
reduced_lane_num 4
b8a28c17-5befd35a
lane_num 1
reduced_lane_num 1
c723ad21-efed33e5
lane_num 3
reduced_lane_num 3
c5eb7f02-4de68c54
lane_num 1
reduced_lane_num 1
bc67b362-fb6767aa
lane_num 2
reduced_lane_num 2
c0c821a8-361cc57e
lane_num 5
reduced_lane_num 4
b7bcc17a-8838a45e
lane_num 0
reduced_lane_num 0
beb87fd7-6120b637
lane_num 1
reduced_lane_num 1
b6ba23a1-4d9e1f20
lane_num 0
reduced_lane_num 0
c9a41ce3-bd04c1ae
lane_num 4
reduced_lane_num 4
b39fe3cd-398678c5
lane_num 1
reduced_lane_num 1
b2d502aa-f8382905
lane_num 0
reduced_lane_num 0
bf293835-27fe9270
lane_num 0
reduced_lane_num 0
c7e6d5d1-c8af9fe3
lane_num 2
reduced_lane_num 2
b851c0af-630d8381
lane_num 0
reduced_lane_num 0
bab692c9-efde0b04
lane_num 4
reduced_lane_num 4
c9df3b7e-0bf4fa31
lane_num 3
reduced_lane_num 3
fe189115-9cc4a501
lane_num 2
reduced_lane_num 2
c09b6e1f-e

lane_num 4
reduced_lane_num 4
b9e91422-2d610458
lane_num 0
reduced_lane_num 0
c91a84db-a199dd0d
lane_num 0
reduced_lane_num 0
b5345bb2-a1424882
lane_num 0
reduced_lane_num 0
b86413b9-c5b79892
lane_num 4
reduced_lane_num 4
c837d2ef-25f73789
lane_num 4
reduced_lane_num 4
c5882339-42bfd7a8
lane_num 3
reduced_lane_num 3
c1dc5cd2-0fd8be03
lane_num 3
reduced_lane_num 3
c9fea009-ee6a4d2a
lane_num 8
reduced_lane_num 4
b8afaffa-5ec67bbb
lane_num 0
reduced_lane_num 0
b8ca6c5d-22de751a
lane_num 2
reduced_lane_num 2
c9cc3cb0-b0085030
lane_num 2
reduced_lane_num 2
c59500f0-130d7415
lane_num 7
reduced_lane_num 4
bd3bd3ab-8e38f822
lane_num 3
reduced_lane_num 3
b7559b8d-8c71431d
lane_num 4
reduced_lane_num 4
bdae005f-d0bb4101
lane_num 2
reduced_lane_num 2
be80a253-75e6f6aa
lane_num 0
reduced_lane_num 0
b26309ce-447c0ac7
lane_num 2
reduced_lane_num 2
b85b0853-ea43f17c
lane_num 2
reduced_lane_num 2
bb46a3b5-6fb47078
lane_num 6
reduced_lane_num 4
c19aa2da-25679033
lane_num 0
reduced_lane_num 0
c87a634a-b

In [10]:
# 拆分数据集
with open(save_path + "/all_with_lane.txt", "r") as f:
    all_list = f.readlines()
sub_train_list = np.random.choice(all_list, 2000, replace=False)
for data in all_list:
    if data not in sub_train_list:
        with open(save_path + "/new_test.txt", "a") as f:
            f.write(data)
    else:
        with open(save_path + "/new_train.txt", "a") as f:
            f.write(data)

In [17]:
# 生成list文件
# img_path = "/home/pantengteng/datasets/bdd100k/images/100k/train"
img_path = "/home/pantengteng/datasets/bdd100k/images/100k/val"
img_list = Path(img_path).glob("*.jpg")
for img in img_list:
    with open("/home/pantengteng/datasets/bdd100k/val.txt","a") as f:
        f.write(img.stem+"\n")

In [30]:
# 可视化label和图片
imgfile = 'image.jpg'
pngfile = 'mask.png'

img = cv2.imread(imgfile, 1)
mask = cv2.imread(pngfile, 0)

contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(img, contours, -1, (0, 0, 255), 1)

img = img[:, :, ::-1]
img[..., 2] = np.where(mask == 1, 255, img[..., 2])

plt.imshow(img)
plt.show()

TypeError: 'NoneType' object is not subscriptable

In [3]:
# 测试for的遍历
a = list(range(1,7))
for x in a:
    x += 1
print(a)
for i in range(len(a)):
    a[i] += 1
print(a)

[1, 2, 3, 4, 5, 6]
[2, 3, 4, 5, 6, 7]


In [7]:
# sort 测试 
a = {"1":[[2,3,5],[2,5,4]],"2":[[3,1,7],[4,1,8]]}
print(a)
b = a["1"]
a["1"].sort(key=lambda x:x[2])
# b.sort()
print(a,b)

{'1': [[2, 3, 5], [2, 5, 4]], '2': [[3, 1, 7], [4, 1, 8]]}
{'1': [[2, 5, 4], [2, 3, 5]], '2': [[3, 1, 7], [4, 1, 8]]} [[2, 5, 4], [2, 3, 5]]


TypeError: list indices must be integers or slices, not tuple

In [6]:
def f(a_dict):
    a_dict["test"] = "test"

In [7]:
test = {"a":"ttt"}
f(test)
print(test)

{'a': 'ttt', 'test': 'test'}
